In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import open3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def drawlines(img1, img2, lines, pts1, pts2):
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 1)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
    return img1, img2


def draw_epipolar(img1, img2, F, pts1, pts2):
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1, img2, lines1, pts1, pts2)

    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2, img1, lines2, pts2, pts1)

    plt.subplot(121)
    plt.imshow(img5)
    plt.subplot(122)
    plt.imshow(img3)
    plt.show()


def visualize_pcd(points):
    """
    Visualize the point cloud.
    """
    pcd = open3d.PointCloud()
    pcd.points = open3d.Vector3dVector(points)
    # Uncommend this line if you want to paint some color
    # pcd.paint_uniform_color([0, 0, 1])
    open3d.draw_geometries([pcd])

In [21]:
def fit_projection(X, Y):    
    # build matrix A based on the correspondences
    A = []
    for i in range(X.shape[0]):
        x, y, z = X[i, 0], X[i, 1], X[i, 2]
        u, v = Y[i, 0], Y[i, 1]
        # equation for each correspondence
        A.append([x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z])
        A.append([0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z])
        
    A = np.array(A)
    
    # solve Ah = 0 using SVD
    U, S, V = np.linalg.svd(A)
    
    # solution h is the smallest singular value of V
    h = V[-1, :]
    
    # build H matrix & normalization
    h = np.append(h, [1])
    H = h.reshape(3, 4)
    # H = H / H[2, 2]
    
    return H

In [22]:
"""
Q1: Camera Calibration
"""
points_2d = np.loadtxt("data/pts2d-norm-pic.txt")
print(points_2d.shape)
# b = np.append(points_2d, np.ones([points_2d.shape[0], 1], dtype=np.int32), axis=1)
# print(b.shape)

points_3d = np.loadtxt("data/pts3d-norm.txt")
print(points_3d.shape)
# A = np.append(points_3d, np.ones([points_3d.shape[0], 1], dtype=np.int32), axis=1)
# print(A.shape)

# P = A @ np.linalg.inv(A.T @ A) @ A.T
P = fit_projection(points_3d, points_2d)
print(P.shape)
print(f"\nP =\n{P}")

(20, 2)
(20, 3)
(3, 4)

P =
[[-2.16139798e-02 -1.99419043e-01  7.24945907e-01  7.02678089e-02]
 [-1.47733446e-02  6.40074852e-02 -2.53073904e-01 -7.28805413e-04]
 [ 2.58411448e-03 -2.38552804e-01  5.51366021e-01  1.00000000e+00]]
